In [1]:
import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display
import gradio as gr 


In [ ]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')


if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")


if openrouter_api_key:
    print(f"OpenRouter API Key exists and begins {openrouter_api_key[:3]}")
else:
    print("OpenRouter API Key not set (and this is optional)")
    

In [ ]:
# Connect to OpenAI client library
# A thin wrapper around calls to HTTP endpoints

openai = OpenAI()

# For Gemini, DeepSeek and Groq, we can use the OpenAI python client
# Because Google and DeepSeek have endpoints compatible with OpenAI
# And OpenAI allows you to change the base_url


openai_client = OpenAI(api_key=openai_api_key)
openrouter_client = OpenAI(api_key=openrouter_api_key, base_url="https://openrouter.ai/api/v1")
ollama_client = OpenAI(api_key="ollama", base_url="http://localhost:11434/v1")
gemini_client = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")


In [ ]:
requests.get("http://localhost:11434/").content


In [ ]:
AGENTS = [
    {
        "name": "Alex",
        "provider": "openai",
        "model": "gpt-4.1-mini",
        "system": "You are Alex. You are argumentative, skeptical, and slightly snarky. You challenge claims directly."
    },
    {
        "name": "Blake",
        "provider": "openrouter",
        "model": "openrouter/aurora-alpha",
        "system": "You are Blake. You are polite and cooperative. You look for common ground and de-escalate tension."
    },
    {
        "name": "Charles",
        "provider": "ollama",
        "model": "llama3.2",
        "system": "You are Charles. You are naive, literal, and often miss the point. You drift into misunderstandings."
    }
]

CLIENTS = {
    "openai": openai_client,
    "openrouter": openrouter_client,
    "ollama": ollama_client,
}

def validate_config(agents):
    providers = {a["provider"] for a in agents}
    if "openai" in providers and not openai_api_key:
        raise ValueError("OPENAI_API_KEY is required for provider 'openai'")
    if "openrouter" in providers and not openrouter_api_key:
        raise ValueError("OPENROUTER_API_KEY is required for provider 'openrouter'")

validate_config(AGENTS)

In [ ]:
def build_messages(current_agent, transcript):
    messages = [{"role": "system", "content": current_agent["system"]}]

    for turn in transcript:
        speaker = turn["speaker"]
        text = turn["text"]

        if speaker == current_agent["name"]:
            messages.append({"role": "assistant", "content": text})
        else:
            messages.append({"role": "user", "content": f"{speaker}: {text}"})

    return messages

def generate_reply(agent, transcript, temperature=0.7):
    client = CLIENTS[agent["provider"]]
    messages = build_messages(agent, transcript)

    response = client.chat.completions.create(
        model=agent["model"],
        messages=messages,
        temperature=temperature
    )

    return (response.choices[0].message.content or "").strip()

def generate_reply_stream(agent, transcript, temperature=0.7):
    client = CLIENTS[agent["provider"]]
    messages = build_messages(agent, transcript)

    stream = client.chat.completions.create(
        model=agent["model"],
        messages=messages,
        temperature=temperature,
        stream=True
    )

    full_text = ""
    for chunk in stream:
        if not getattr(chunk, "choices", None):
            continue
        delta = chunk.choices[0].delta
        if delta is None:
            continue
        piece = delta.content or ""
        if piece:
            full_text += piece
            yield full_text

def run_conversation(seed_prompt, rounds=3, verbose=True):
    transcript = [{"speaker": "User", "text": seed_prompt}]

    for _ in range(rounds):
        for agent in AGENTS:
            reply = generate_reply(agent, transcript)
            transcript.append({"speaker": agent["name"], "text": reply})

            if verbose:
                print(f"{agent['name']}: {reply}\n")

    return transcript

def run_conversation_stream(seed_prompt, rounds=3, verbose=False):
    transcript = [{"speaker": "User", "text": seed_prompt}]
    yield transcript

    for _ in range(rounds):
        for agent in AGENTS:
            transcript.append({"speaker": agent["name"], "text": ""})

            for partial_reply in generate_reply_stream(agent, transcript[:-1]):
                transcript[-1]["text"] = partial_reply
                yield transcript

            if not transcript[-1]["text"]:
                transcript[-1]["text"] = "[No response returned]"
                yield transcript

            if verbose:
                print(f"{agent['name']}: {transcript[-1]['text']}\\n")

def render_markdown_transcript(transcript):
    parts = ["## Conversation Transcript"]
    for turn in transcript:
        parts.append(f"**{turn['speaker']}**: {turn['text']}")
    display(Markdown("\n\n".join(parts)))

In [ ]:
#seed = "Debate whether AI will improve education in the next 5 years."
#transcript = run_conversation(seed_prompt=seed, rounds=2, verbose=True)
#render_markdown_transcript(transcript)

In [ ]:
def transcript_to_markdown(transcript):
    parts = ["## Conversation Transcript"]
    for turn in transcript:
        parts.append(f"**{turn['speaker']}**: {turn['text']}")
    return "\n\n".join(parts)

def debate_ui_stream(seed_prompt):
    try:
        for transcript in run_conversation_stream(seed_prompt=seed_prompt, rounds=2, verbose=False):
            yield transcript_to_markdown(transcript)
    except Exception as e:
        yield f"## Error\n\n{type(e).__name__}: {e}"

view = gr.Interface(
    fn=debate_ui_stream,
    title="Model Debates",
    inputs=gr.Textbox(label="Proposed Topic", info="Enter a debate topic", lines=4),
    outputs=gr.Markdown(label="Response"),
    examples=[["Debate whether AI will improve education in the next 5 years."]],
    flagging_mode="never"
)

view.queue().launch(share=True)